# Initialisation

## Preinitialisation

In [ ]:
data = "Div"

In [ ]:
%matplotlib inline

path_to_protein = "./proteins_Hum" + data + "_bg.txt"

import numpy as np
import pandas as pd
import scipy.stats as st
from math import *
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import *
from sklearn.neural_network import MLPClassifier
%run Functions.py
from deep_forest_utils import CascadeForest
import random
class ProbRandomForestClassifier(RandomForestClassifier):
    def predict(self, X):
        return RandomForestClassifier.predict_proba(self, X)

In [ ]:
def reading(path, y_read = True):   
    Datap = pd.read_csv(path, delimiter=",")
    if y_read:
        X, y = Datap.iloc[:, :(Datap.shape[1] - 1)], Datap.iloc[:, Datap.shape[1] - 1]
    else:
        X = Datap
    continue_inds = (Datap.dtypes == np.float64) | (Datap.dtypes == np.int64)
    X_float = X.ix[:, continue_inds]
    X_factor = X.ix[:, np.invert(continue_inds)]
    imp = Imputer(missing_values="NaN", strategy="median", axis=0)
    X_full = pd.DataFrame(imp.fit_transform(X_float))
    X_full.columns = X_float.columns
    X_full.index = X_float.index
    if y_read:
        return X_full, y
    else:
        return X_full

In [ ]:
def cv_wo_protein_intersection(path_to_acc = path_to_protein, n_cv = 5):
    acc = pd.read_csv(path_to_acc, delimiter=",")
    classnames, indices = np.unique(np.array(acc), return_inverse=True)
    kf = KFold(n_splits= n_cv, shuffle=True)
    acc_split = list(kf.split(np.arange(len(classnames))))
    pdind = pd.Series(indices)
    acc_ind = pd.Series(np.arange(len(acc)))
    return [(np.array(acc_ind[pdind.isin(acc_split[i][0])]),
             np.array(acc_ind[pdind.isin(acc_split[i][1])]))
            for i in range(n_cv)]

In [ ]:
def roc_plot(data_name, fpr, tpr, roc_auc, acc, lw = 2, n_clf = 9, save = False):
    # -------- sorting --------
    values = roc_auc
    ind = Series(roc_auc).sort_values(ascending=False).index
    # ----------------------------
    colors = ['black', 'darkred', 'red', 'orange', 'gold', 'lime', 'green', 'blue', 'purple']
    clf_names = ['Deep Forest', 'Neural network', 'Logistic Regression', 'Gaussian NB (GNB)', 
                 'Random Forest', 'Boosted GNB', 'Gaussian SVM', 'Linear SVM', 'Polynomial SVM'] 
    for i, col in zip(ind, colors):
        plt.plot(fpr[i], tpr[i], color=col,
             lw=lw, label="{}: AUC = {: .3f}, acc = {: .3f}".format(clf_names[i], roc_auc[i], acc[i]))
    sns.set_style("whitegrid")
    sns.set_context("talk")
    plt.plot([0, 1], [0, 1], color='grey', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.003])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Roc curves of Hum' + data_name + ' dataset')
    plt.legend(loc="lower right")
    plt.xticks(np.arange(0., 1., 0.1))
    plt.yticks(np.arange(0., 1., 0.1))
    plt.rcParams["figure.figsize"] = [11,11]
    if save == True:
        plt.savefig('./Hum' + data_name + '_roc.png')
        plt.savefig('./Hum' + data_name + '_roc.pdf')
        plt.savefig('./Hum' + data_name + '_roc.svg')

## Reading

In [ ]:
Xvar, yvar = reading("./Data_HumVar_bg.txt")
Xdiv, ydiv = reading("./Data_HumDiv_bg.txt")
Xd, yd = reading("./Data_Dog_bg.txt")
Xm, ym = reading("./Data_Mouse.txt")

## Classifiers parameters

In [ ]:
if data == "Var":
    svp = SVC(C=6000, coef0=0.01, kernel='poly')
    gnb = GaussianNB()
    svr = SVC(C = 10000, kernel='rbf')
    svm = LinearSVC(C=8)
    lr = LogisticRegression(C = 5)
    rf = RandomForestClassifier(n_estimators=2500)
    bnb = AdaBoostClassifier(base_estimator=gnb, n_estimators=50, algorithm='SAMME.R')
    nn = MLPClassifier(solver='adam', alpha=0.1, hidden_layer_sizes=(30, 70, 10), random_state=1)

elif data == "Div":
    svp = SVC(C=9000, coef0=0.1, kernel='poly')
    gnb = GaussianNB()
    svr = SVC(C = 18000, kernel='rbf')
    svm = LinearSVC(C=17)
    lr = LogisticRegression(C = 130)
    rf = RandomForestClassifier(n_estimators=1200)
    bnb = AdaBoostClassifier(base_estimator=gnb, n_estimators=3, algorithm='SAMME.R')
    nn = MLPClassifier(solver='adam', alpha=0.01, hidden_layer_sizes=(90, 20), random_state=1)


# Inductive transfer learning

In [ ]:
Xmammal, ymammal = Xd, yd

In [ ]:
# Definition of weights
if data == "Div":
    X_wo_bias = Xdiv
    y_train = ydiv
elif data == "Var":
    X_wo_bias = Xvar
    y_train =yvar
dif = X_wo_bias.median() - Xmammal.median()
X_train = X_wo_bias - dif
wt0 = X_train.apply(lambda row: ((Xmammal.mean() - row)**2).sum(), axis = 1)
wt = [exp( - x) for x in wt0]

In [ ]:
clf_names = ['Logistic Regression', 'Random Forest', 'Linear SVM', 'Polynomial SVM', 'Gaussian SVM']
clfs = [lr, rf, svm, svp, svr]
for clf, name in zip(clfs, clf_names):
    clf.fit(X_wo_bias, y_train, sample_weight=wt)
    y_pred = clf.predict(Xmammal)
    score = accuracy_score(ymammal, y_pred)
    print("Accuracy of %s = %.3f (with weights)" % (name, score))
    clf.fit(X_wo_bias, y_train)
    score = clf.score(Xmammal, ymammal)
    print("Accuracy of %s = %.3f (wo weights)" % (name, score))

# Visualisation

## Computations

In [ ]:
if data == "Var":
    X_train = Xvar
    y_train = yvar
    svp = SVC(C=6000, coef0=0.01, kernel='poly')
    gnb = GaussianNB()
    svr = SVC(C = 10000, kernel='rbf')
    svm = LinearSVC(C=8)
    lr = LogisticRegression(C = 5)
    rf = RandomForestClassifier(n_estimators=2500)
    bnb = AdaBoostClassifier(base_estimator=gnb, n_estimators=50, algorithm='SAMME.R')
    nn = MLPClassifier(solver='adam', alpha=0.1, hidden_layer_sizes=(30, 70, 10), random_state=1)

elif data == "Div":
    X_train = Xdiv
    y_train = ydiv
    svp = SVC(C=9000, coef0=0.1, kernel='poly')
    gnb = GaussianNB()
    svr = SVC(C = 18000, kernel='rbf')
    svm = LinearSVC(C=17)
    lr = LogisticRegression(C = 130)
    rf = RandomForestClassifier(n_estimators=1200)
    bnb = AdaBoostClassifier(base_estimator=gnb, n_estimators=3, algorithm='SAMME.R')
    nn = MLPClassifier(solver='adam', alpha=0.01, hidden_layer_sizes=(90, 20), random_state=1)

In [ ]:
my_cv = cv_wo_protein_intersection()

In [ ]:
fpr, thr, tpr, roc_auc, acc = [], [], [], [], []
bnb = AdaBoostClassifier(base_estimator=gnb, n_estimators=50, algorithm='SAMME')

N_estimators = 1000
cascade_forest_params1 = RandomForestClassifier(n_estimators=N_estimators,min_samples_split=11,max_features=1,
                                                n_jobs=-1).get_params()
cascade_forest_params2 = RandomForestClassifier(n_estimators=N_estimators,min_samples_split=11,max_features='sqrt',
                                                n_jobs=-1).get_params()
cascade_params_list = [cascade_forest_params1,cascade_forest_params2]*2
drf = CascadeForest(ProbRandomForestClassifier(),cascade_params_list,k_fold=my_cv)

for clf in [drf]:
    print(clf)
    clf.fit(X_train, y_train)
    score = clf.score
    print("Accurancy = %.4f" % score)
    acc = acc + [score]
    y_pred = clf.predict_proba_cv(X_train, y_train)
    metr = metrics.roc_curve(y_train, y_pred[:,1])
    fpr = fpr + [metr[0]]
    tpr = tpr + [metr[1]]
    roc_auc = roc_auc + [metrics.auc(metr[0], metr[1])]
    
for clf in [nn, lr, gnb, rf, bnb]:
    print(clf)
    score = cross_val_score(clf, X_train, y_train, cv=5, n_jobs=50).mean()
    print("Accurancy = %.4f" % score)
    acc = acc + [score]
    y_pred = cross_val_predict(clf, X_train, y_train, cv=my_cv, method="predict_proba", n_jobs=50)
    metr = metrics.roc_curve(y_train, y_pred[:, 1])
    fpr = fpr + [metr[0]]
    tpr = tpr + [metr[1]]
    roc_auc = roc_auc + [metrics.auc(metr[0], metr[1])]

for clf in [svr, svm, svp]:
    print(clf)
    score = cross_val_score(clf, X_train, y_train, cv=my_cv, n_jobs=50).mean()
    print("Accurancy = %.4f" % score)
    acc = acc + [score]
    y_pred = cross_val_predict(clf, X_train, y_train, cv=my_cv, method="decision_function", n_jobs=50)
    y_pred = (y_pred - min(y_pred)) / (max(y_pred) - min(y_pred))
    metr = metrics.roc_curve(y_train, y_pred)
    fpr = fpr + [metr[0]]
    tpr = tpr + [metr[1]]
    roc_auc = roc_auc + [metrics.auc(metr[0], metr[1])]

In [ ]:
if data == "Var":
    fprvar = fpr
    tprvar = tpr
    roc_aucvar = roc_auc
    accvar = acc
elif data == "Div":
    fprdiv = fpr
    tprdiv = tpr
    roc_aucdiv = roc_auc
    accdiv = acc

## Roc curves

In [ ]:
# run at least two times to obtain nice plot
roc_plot(data, fprdiv, tprdiv, roc_aucdiv, accdiv, lw = 2, n_clf = 9, save = False)

In [ ]:
ind = Series(roc_auc).sort_values(ascending=False).index
clf_names = ['Deep Forest', 'Neural network', 'Logistic Regression', 'Gaussian NB (GNB)', 
                 'Random Forest', 'Boosted GNB', 'Gaussian SVM', 'Linear SVM', 'Polynomial SVM']
for i in range(9):
    fp = fprvar[ind[i]]
    tp = tprvar[ind[i]]
    #print(clf_names[ind[i]])
    print('{: .3f}'.format(accvar[ind[i]]))
    print(' '.join(['%0.3f' % n for n in [tp[(fp > x - 0.005) & (fp < x + 0.005)].mean() for x in [0.05, 0.1, 0.2]]]))